# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Data Analysis

This section of feature selection was done in the data_preprocessing notebook, but because it doesn't take too long to retrieve the features dictionary, we re-added it in this analysis.

Used LASSO regression to select features. 

Retrospectively, this may not be necessary, as we apply GridSearchCV later across L1, L2, and Elasticnet penalties, meaning the GridSearch would automatically feature select using L1 based on cross-validated accuracy.

Still, it can speed up computation time by initially getting rid of seemingly useless features.

In [8]:
X_train_full = pd.read_csv('X_train_full.csv')
X_test_full = pd.read_csv('X_test_full.csv')
y_train_bin = pd.read_csv('y_train_bin.csv')['is_obese']
y_test_bin = pd.read_csv('y_test_bin.csv')['is_obese']
y_test_multi = pd.read_csv('y_test_multi.csv')['obesity_category']
y_train_multi = pd.read_csv('y_train_multi.csv')['obesity_category']

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
c_arr = [0.1, 1, 10]

features_dict = {}

# For each regularization strength, apply LASSO and retrieve non-zero features
for C in c_arr:
    lr = LogisticRegression(penalty="l1", solver="saga", C=C, multi_class="multinomial", max_iter=10000)
    lr.fit(X_train_full, y_train_multi)

    features_dict[C] = X_train_full.columns[lr.coef_[0] != 0]

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [13]:
# 0.1 = strongest regularization, most minimal set of features
# 1 = second strongest regularization, moderate set of features
# 10 = least regularization, most features

features_dict

{0.1: Index(['FCVC^2', 'FCVC NCP', 'FCVC CH2O', 'NCP FAF', 'NCP TUE', 'log1p_Age',
        'log1p_Weight', 'exp_NCP', 'Gender_Male', 'CAEC_Frequently'],
       dtype='object'),
 1: Index(['Age', 'Weight', 'Age^2', 'Age Weight', 'FCVC^2', 'FCVC CH2O',
        'FCVC FAF', 'FCVC TUE', 'FCVC Weight', 'NCP FAF', 'NCP TUE',
        'NCP Weight', 'CH2O FAF', 'CH2O TUE', 'TUE^2', 'log1p_FCVC',
        'log1p_CH2O', 'log1p_TUE', 'log1p_Weight', 'exp_FCVC', 'exp_CH2O',
        'exp_FAF', 'Gender_Male', 'FAVC_yes', 'CAEC_Frequently',
        'CAEC_Sometimes', 'SCC_yes', 'CALC_Frequently',
        'MTRANS_Public_Transportation'],
       dtype='object'),
 10: Index(['Age', 'Weight', 'FAF', 'TUE', 'Age^2', 'Age FCVC', 'Age NCP',
        'Age CH2O', 'Age FAF', 'Age Weight', 'FCVC^2', 'FCVC NCP', 'FCVC CH2O',
        'FCVC FAF', 'FCVC TUE', 'FCVC Weight', 'NCP^2', 'NCP CH2O', 'NCP FAF',
        'NCP TUE', 'NCP Weight', 'CH2O^2', 'CH2O FAF', 'CH2O TUE',
        'CH2O Weight', 'FAF^2', 'FAF TUE', 'FAF W

# Regressions

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

## Logistic Regression

### Not Regularized

We use 5-fold cross validation to generated cross-validated accuracies for each possible set of features.

Using 5-fold cross validation is also consistent with how we conduct GridSearch, allowing us to see the improvement finding optimal (over the grid) regularization parameters has on the model. 

For example, for "C used for feature selection: 1", the CV accuracy goes from ~0.948 to ~0.951 - this increase should be attributable to the regularization parameters chosen by GridSearch, since we use the same evaluation protocol for unregularized and regularized. When rerunning this multiple times, the CV accuracy has changed, but the general trend is that the CV accuracy when C increases.

In [21]:
cv_scores = {}

for c, features in features_dict.items():
    lr = LogisticRegression(max_iter=100000)

    scores = cross_val_score(
        lr,
        X_train_full[features],
        y_train_bin,
        cv=5,
        scoring=make_scorer(accuracy_score)
    )

    cv_scores[c] = scores

    print(f"C used for feature selection: {c}")
    print(f"CV accuracy: {np.mean(scores)}\n")


C used for feature selection: 0.1
CV accuracy: 0.9407599248503151

C used for feature selection: 1
CV accuracy: 0.9460871244710551

C used for feature selection: 10
CV accuracy: 0.9496461994978315



From the above, for unregularized multinomial logistic regression, using the largest set of features gives the greatest validation accuracy.

### Regularization Gridsearch

Note that we use "saga" solver because it has support for all penalty types (and multinomial logistic regression as well). 

"sag" and "saga" solvers only guaranteed to converge with features with the same scale according to scikit-learn documentation. This is accounted for during our pre-processing stage.

GridSearch goes over all possible combinations of features provided in the parameter grid and outputs the one with the highest 5-fold cross-validation accuracy.

In [27]:
# NOTE: Runtime for this cell > 2 minutes.
# Outputs in markdown below for reproducibility of optimal models

param_grid = [
    {
        'penalty': ['l1'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['saga'],
    },
    {
        'penalty': ['l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['saga'],
    },
    {
        'penalty': ['elasticnet'],
        'C': [0.01, 0.1, 1, 10],
        'l1_ratio': [0.25, 0.5, 0.75],
        'solver': ['saga'],
    }
]

binary_grid_dict = {}

for c, features in features_dict.items():
    model = LogisticRegression(max_iter=100000, solver="saga")
    grid = GridSearchCV(model, param_grid, cv=5)
    grid.fit(X_train_full[features], y_train_bin)

    binary_grid_dict[c] = grid

    print("C used for feature selection: ", c)
    print(grid.best_params_)
    print(grid.best_score_, "\n")

C used for feature selection:  0.1
{'C': 1, 'l1_ratio': 0.25, 'penalty': 'elasticnet', 'solver': 'saga'}
0.9419451126367356 

C used for feature selection:  1
{'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.9520130634031571 

C used for feature selection:  10
{'C': 1, 'l1_ratio': 0.75, 'penalty': 'elasticnet', 'solver': 'saga'}
0.9526100468807612 



C used for feature selection:  0.1
{'C': 1, 'l1_ratio': 0.25, 'penalty': 'elasticnet', 'solver': 'saga'}
0.9419451126367356 

C used for feature selection:  1
{'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.9520130634031571 

C used for feature selection:  10
{'C': 1, 'l1_ratio': 0.75, 'penalty': 'elasticnet', 'solver': 'saga'}
0.9526100468807612 

Optimal parameters: C used for feature selection:  10, {'C': 1, 'l1_ratio': 0.75, 'penalty': 'elasticnet', 'solver': 'saga'}

## Multinomial Regression

### Not Regularized

In [34]:
# NOTE: Runtime for this cell > 5 minutes.
# Outputs in markdown below for reproducibility of optimal models
cv_scores = {}

for c, features in features_dict.items():
    lr = LogisticRegression(max_iter=100000, solver = 'saga')

    scores = cross_val_score(
        lr,
        X_train_full[features],
        y_train_multi,
        cv=5,
        scoring=make_scorer(accuracy_score)
    )

    cv_scores[c] = scores

    print(f"C used for feature selection: {c}")
    print(f"CV accuracy: {np.mean(scores)}\n")

C used for feature selection: 0.1
CV accuracy: 0.7802293118887504

C used for feature selection: 1
CV accuracy: 0.8264393447228416

C used for feature selection: 10
CV accuracy: 0.8429986128913315



C used for feature selection: 0.1
CV accuracy: 0.7802293118887504

C used for feature selection: 1
CV accuracy: 0.8424086527487578

C used for feature selection: 10
CV accuracy: 0.838268396748196

From the above, for unregularized multinomial logistic regression, using the largest set of features gives the greatest validation accuracy.

### Regularization Gridsearch

For this section we reduce the size of the Gridsearch, and lower the maximum iteration limit, due to computational constraints.

In [ ]:
# NOTE: Runtime for this cell > 10 minutes.
# Outputs in markdown below for reproducibility of optimal models

param_grid = [
    {
        'penalty': ['l1'],
        'C': [0.1, 1, 10],
        'solver': ['saga'],
    },
    {
        'penalty': ['l2'],
        'C': [0.1, 1, 10],
        'solver': ['saga'],
    },
    {
        'penalty': ['elasticnet'],
        'C': [0.1, 1, 10],
        'l1_ratio': [0.25, 0.5, 0.75],
        'solver': ['saga'],
    }
]

multi_grid_dict = {}

for c, features in features_dict.items():
    model = LogisticRegression(max_iter=10000, solver="saga")
    grid = GridSearchCV(model, param_grid, cv=5)
    grid.fit(X_train_full[features], y_train_multi)

    multi_grid_dict[c] = grid

    print("C used for feature selection: ", c)
    print(grid.best_params_)
    print(grid.best_score_, "\n")

C used for feature selection:  0.1
{'C': 1, 'penalty': 'l1', 'solver': 'saga'}
0.7956279739434271 

C used for feature selection:  1
{'C': 10, 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
0.8317700560110968 



C used for feature selection:  0.1
{'C': 1, 'penalty': 'l1', 'solver': 'saga'}
0.7956279739434271 

C used for feature selection:  1
{'C': 10, 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
0.8317700560110968 

C used for feature selection:  10
{'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.8453777676329605 

Optimal parameters: C used for feature selection:  10, {'C': 10, 'penalty': 'l1', 'solver': 'saga'}

# Test Evaluations

In [ ]:
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, f1_score
)

## Logistic Regression

In [ ]:
final_bin_lr = LogisticRegression(penalty="elasticnet", C=1, l1_ratio=0.75, max_iter=10000, solver="saga")
final_bin_features = features_dict[10]

final_bin_lr.fit(X_train_full[final_bin_features], y_train_bin)

y_pred = final_bin_lr.predict(X_test_full[final_bin_features])

print(confusion_matrix(y_test_bin, y_pred))

print("Accuracy:", accuracy_score(y_test_bin, y_pred))
print("Precision:", precision_score(y_test_bin, y_pred))
print("Recall:", recall_score(y_test_bin, y_pred))
print("F1 Score:", f1_score(y_test_bin, y_pred))

print(classification_report(y_test_bin, y_pred))


In [ ]:
# Regression Coefficients for Binomial Model
coefficients = final_bin_lr.coef_[0]
importance = pd.DataFrame({
    'Variable': X_train_full[final_bin_features].columns,
    'Coefficient': coefficients,
    'Absolute Coefficient': np.abs(coefficients)
})
important_vars = importance.sort_values(by='Absolute Coefficient', ascending=False)

important_vars

In [ ]:
# ROC Curve for Binomial Model
from sklearn.metrics import roc_curve, auc

y_probs = final_bin_lr.predict_proba(X_test_full[final_bin_features])[:, 1]

fpr, tpr, thresholds = roc_curve(y_test_bin, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve - Binary Model')
plt.legend(loc='lower right')
plt.grid()
plt.show()

Not many notes here, on the binary case our chosen logistic regression model has strong performance across the board.

## Multinomial Regression

In [ ]:
final_multi_lr = LogisticRegression(penalty="l1", C=10, max_iter=10000, multi_class="multinomial", solver="saga")
final_multi_features = features_dict[10]

final_multi_lr.fit(X_train_full[final_multi_features], y_train_multi)

y_pred = final_multi_lr.predict(X_test_full[final_multi_features])

print(confusion_matrix(y_test_multi, y_pred))

print("Accuracy:", accuracy_score(y_test_multi, y_pred))

print(classification_report(y_test_multi, y_pred))

Final model is extremely impressive at Obese and Insufficient Weight cases, but struggles on the more moderate Normal Weight and Overweight categories.

Practically speaking, this is not a bad thing. The model should prioritize classifying high risk individuals.

In [ ]:
# Regression Coefficients for Multinomial Model
coefficients = final_multi_lr.coef_[0]
importance = pd.DataFrame({
    'Variable': X_train_full[final_multi_features].columns,
    'Coefficient': coefficients,
    'Absolute Coefficient': np.abs(coefficients)
})
important_vars = importance.sort_values(by='Absolute Coefficient', ascending=False)

important_vars